In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import spacy
import seaborn as sns
from spacy.util import minibatch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_path = "/kaggle/input/sms-spam-collection-dataset/spam.csv"
data = pd.read_csv(data_path, encoding="ISO-8859-1")
print(data.head())

In [ ]:
data = pd.read_csv(data_path, encoding="ISO-8859-1")
observations = len(data.index)
print(f"Size of Dataset: {observations}\n")
print(data['v1'].value_counts())
print()
print(data['v1'].value_counts() / len(data.index) * 100.0)

In [ ]:
nlp = spacy.blank("en")

text_cat = nlp.create_pipe(
            "textcat",
            config={
                "exclusive_classes": True,
                "architecture": "bow"})

nlp.add_pipe(text_cat)

text_cat.add_label("ham")
text_cat.add_label("spam")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
        data['v2'], data['v1'], test_size=0.33, random_state=7)

In [ ]:
train_lables = [{'cats': {'ham': label == 'ham',
                          'spam': label == 'spam'}} for label in y_train]

test_lables = [{'cats': {'ham': label == 'ham',
                          'spam': label == 'spam'}} for label in y_test]

train_data = list(zip(x_train, train_lables))
test_data = list(zip(x_test, train_lables))

In [ ]:
def train_model(model, train_data, optimizer, batch_size, epochs=10):
    losses = {}
    random.seed(1)

    for epoch in range(epochs):
        random.shuffle(train_data)

        batches = minibatch(train_data, size=batch_size)
        for batch in batches:
            
            texts, labels = zip(*batch)

            
            model.update(texts, labels, sgd=optimizer, losses=losses)
        print("Loss: {}".format(losses['textcat']))

    return losses['textcat']

In [ ]:
optimizer = nlp.begin_training()
batch_size = 5
epochs = 3


train_model(nlp, train_data, optimizer, batch_size, epochs)

In [ ]:
print(train_data[0])
sample_test = nlp(train_data[0][0])
print(sample_test.cats)

In [ ]:
def get_predictions(model, texts):
    
    docs = [model.tokenizer(text) for text in texts]

    
    textcat = model.get_pipe('textcat')
    scores, _ = textcat.predict(docs)

    
    predicted_labels = scores.argmax(axis=1)
    predicted_class = [textcat.labels[label] for label in predicted_labels]

    return predicted_class

In [ ]:
train_predictions = get_predictions(nlp, x_train)
test_predictions = get_predictions(nlp, x_test)

train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)

print(f"Train accuracy: {train_accuracy}")
print(f"Test accuracy: {test_accuracy}")

In [ ]:
print("TRAIN:")
cf_train_matrix = confusion_matrix(y_train, train_predictions)
plt.figure(figsize=(10,8))
sns.heatmap(cf_train_matrix, annot=True, fmt='d')

In [ ]:
print("TEST:")
cf_test_matrix = confusion_matrix(y_test, test_predictions)
plt.figure(figsize=(10,8))
sns.heatmap(cf_test_matrix, annot=True, fmt='d')